In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

import matplotlib.pyplot as plt
%matplotlib inline

from InteriorHelmholtzSolver3D import *
from ExampleBoundaries import Sphere

frequency = 20.0 # frequency [Hz]
k = frequencyToWavenumber(frequency)

interiorPoints = np.array([[0.500, 0.000, 0.000],
                           [0.000, 0.000, 0.010],
                           [0.000, 0.000, 0.250],
                           [0.000, 0.000, 0.500],
                           [0.000, 0.000, 0.750]], dtype=np.float32)

# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#

solver = InteriorHelmholtzSolver3D(*(Sphere()))

boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)
boundaryCondition.f[:] = np.sin(k * solver.aCenters[:,2])

boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "Test Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 2
# Neumann boundary condition with v = cos(k*z)
#

boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)

for i in range(solver.aElement.shape[0]):
    a = solver.aVertex[solver.aElement[i, 0]]
    b = solver.aVertex[solver.aElement[i, 1]]
    c = solver.aVertex[solver.aElement[i, 2]]
    normal = Normal3D(a, b, c)
    boundaryCondition.f[i] = k * np.cos(k * solver.aCenters[i,2]) * normal[2]

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 3
# Neumann boundary condition with v = cos(k*z)
#

boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

p = np.array([0.0, 0.0, 0.0], dtype=np.float32)
for i in range(solver.aElement.shape[0]):
    q = solver.aCenters[i, :]
    r = p - q
    R = norm(r)
    boundaryIncidence.phi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)
    a = solver.aVertex[solver.aElement[i, 0]]
    b = solver.aVertex[solver.aElement[i, 1]]
    c = solver.aVertex[solver.aElement[i, 2]]
    drbdn = -np.dot(r, normal) / R
    boundaryIncidence.v[i] = drbdn * np.exp(1.0j * k * R)*(1.0j * k * R - 1.0) / \
            (4.0 * np.pi * R * R)

for i in range(interiorPoints.shape[0]):
    q = interiorPoints[i, :]
    r = p - q
    R = norm(r)
    interiorIncidentPhi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)
